<a href="https://colab.research.google.com/github/Mr-Hackrr/pySpark/blob/main/pspyk10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Create local file with the following data

Complete the following code using pyspark RDDs

movie.csv
movieId,moviename,director,year
200,shawshank,FrankD,1994
201,saw,JamesW,2004
202,avengers,JossW,2012
204,mi,ChristopherMcq,2018
205,wrongturn,DeclanO,2011

award.csv
catalogId,movieId,awards
11,200,2
12,200,3
13,201,1
14,202,3
15,202,5
16,204,4
17,205,1

In [4]:
# prompt: make abouve data into csv files

%%writefile movie.csv
movieId,moviename,director,year
200,shawshank,FrankD,1994
201,saw,JamesW,2004
202,avengers,JossW,2012
204,mi,ChristopherMcq,2018
205,wrongturn,DeclanO,2011



Writing movie.csv


In [5]:
%%writefile award.csv
catalogId,movieId,awards
11,200,2
12,200,3
13,201,1
14,202,3
15,202,5
16,204,4
17,205,1

Writing award.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=a5dcedf0fe3dbe9d5df1d683db8e6dd43616b5e9239e99c9caa8dc0fc032e088
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
#Import libraries & Create SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("movie analysis").getOrCreate()

In [7]:
#Create two dataframes using respective csv files from local storage
movie_data = spark.read.option("header","True").csv("movie.csv")
award_data = spark.read.option("header","True").csv("award.csv")
movie_data.limit(3).show()
award_data.limit(3).show()

+-------+---------+--------+----+
|movieId|moviename|director|year|
+-------+---------+--------+----+
|    200|shawshank|  FrankD|1994|
|    201|      saw|  JamesW|2004|
|    202| avengers|   JossW|2012|
+-------+---------+--------+----+

+---------+-------+------+
|catalogId|movieId|awards|
+---------+-------+------+
|       11|    200|     2|
|       12|    200|     3|
|       13|    201|     1|
+---------+-------+------+



In [16]:
#Create a new column called "movieInfo" in movies dataframe which includes movieName & Director
#from pyspark.sql.functions import concat, lit
movie_data.withColumn("movieinfo",concat(movie_data.moviename,lit(" "),movie_data.director)).show()


+-------+---------+--------------+----+-----------------+
|movieId|moviename|      director|year|        movieinfo|
+-------+---------+--------------+----+-----------------+
|    200|shawshank|        FrankD|1994| shawshank FrankD|
|    201|      saw|        JamesW|2004|       saw JamesW|
|    202| avengers|         JossW|2012|   avengers JossW|
|    204|       mi|ChristopherMcq|2018|mi ChristopherMcq|
|    205|wrongturn|       DeclanO|2011|wrongturn DeclanO|
+-------+---------+--------------+----+-----------------+



In [58]:
#Calculate total number of awards for each movie & display - movieID, movienName, awardsCount
full_data = movie_data.join(award_data,movie_data.movieId == award_data.movieId)
full_data.groupBy(movie_data.moviename,movie_data.movieId).agg({'awards':'sum'}).show()


+---------+-------+-----------+
|moviename|movieId|sum(awards)|
+---------+-------+-----------+
|wrongturn|    205|        1.0|
|shawshank|    200|        5.0|
|       mi|    204|        4.0|
|      saw|    201|        1.0|
| avengers|    202|        8.0|
+---------+-------+-----------+



In [63]:
#Get top three movies on the basis of number of awards
full_data = movie_data.join(award_data,movie_data.movieId == award_data.movieId)
new_data = full_data.groupBy(movie_data.moviename,movie_data.movieId).agg({'awards':'sum'}).withColumnRenamed("sum(awards)","awardCount")
new_data.sort(col("awardCount").desc()).limit(3).show()

+---------+-------+----------+
|moviename|movieId|awardCount|
+---------+-------+----------+
| avengers|    202|       8.0|
|shawshank|    200|       5.0|
|       mi|    204|       4.0|
+---------+-------+----------+



In [67]:
#Get top three latest movies
movie_data.orderBy(col('year').desc()).limit(3).show()

+-------+---------+--------------+----+
|movieId|moviename|      director|year|
+-------+---------+--------------+----+
|    204|       mi|ChristopherMcq|2018|
|    202| avengers|         JossW|2012|
|    205|wrongturn|       DeclanO|2011|
+-------+---------+--------------+----+

